# 추가한 데이터셋을 기존 데이터셋에 추가

라벨링하여 export 된 라벨링 데이터셋이 있는 경로를 지정하면,
train/val 로 분할하여 복사된 이미지와 매칭되는 라벨링 파일이 저장됨

In [ ]:
import os
import shutil
import random

# ──────────────────────────────────────
# 0) 환경에 맞게 경로만 수정
# ──────────────────────────────────────
SRC_IMG_DIR  = "tree/Prediction/tree_0_detected"    # 추가할 원본 이미지 폴더
# 수동 라벨링된 .txt 파일들이 모두 들어있는 폴더
# (train/val 구분 없이 한 곳에 몰려있음)
SRC_LBL_DIR  = "tree/Datasets/0_tree_detected_5/labels/train" # 이곳 수정!!!

DST_IMG_ROOT = "tree/Datasets/250630_split/images" # 기존 train/val 상위
DST_LBL_ROOT = "tree/Datasets/250630_split/labels"
TRAIN_RATIO  = 0.8

# ──────────────────────────────────────
# 1) 원본 파일 목록 준비
# ──────────────────────────────────────
all_images = [
    f for f in os.listdir(SRC_IMG_DIR)
    if f.lower().endswith((".jpg", ".png"))
]
random.seed(42)
random.shuffle(all_images)

split_idx  = int(len(all_images) * TRAIN_RATIO)
train_imgs = all_images[:split_idx]
val_imgs   = all_images[split_idx:]

print(f"추가할 이미지 총 {len(all_images)}장 → train: {len(train_imgs)}장, val: {len(val_imgs)}장")

# ──────────────────────────────────────
# 2) 안전 복사 함수
# ──────────────────────────────────────
def safe_copy(src, dst):
    if not os.path.isfile(src):
        print(f"⚠️ 원본 누락: {src}")
        return False
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    if os.path.exists(dst):
        print(f"⚠️ 이미 존재: {dst} (스킵)")
        return False
    shutil.copy2(src, dst)
    return True

# ──────────────────────────────────────
# 3) Dry-run vs 실제 실행 설정
# ──────────────────────────────────────
dry_run = True  # True면 실제 복사 없이 로그만 출력. 완료 후 False로 변경.

# ──────────────────────────────────────
# 4) train/val 폴더에 복사
# ──────────────────────────────────────
for split, img_list in [("train", train_imgs), ("val", val_imgs)]:
    dst_img_dir = os.path.join(DST_IMG_ROOT, split)
    dst_lbl_dir = os.path.join(DST_LBL_ROOT, split)

    for img_name in img_list:
        # 원본 이미지 경로
        src_img = os.path.join(SRC_IMG_DIR, img_name)
        dst_img = os.path.join(dst_img_dir, img_name)

        # 원본 라벨 경로 (단일 SRC_LBL_DIR에서 가져옴)
        lbl_name = os.path.splitext(img_name)[0] + ".txt"
        src_lbl   = os.path.join(SRC_LBL_DIR, lbl_name)
        dst_lbl   = os.path.join(dst_lbl_dir, lbl_name)

        if dry_run:
            print(f"[DRY-RUN] 이미지 → {src_img} -> {dst_img}")
            print(f"[DRY-RUN] 레이블 → {src_lbl} -> {dst_lbl}")
        else:
            ok_img = safe_copy(src_img, dst_img)
            ok_lbl = safe_copy(src_lbl, dst_lbl)
            if ok_img and ok_lbl:
                print(f"✅ {split}에 추가된 파일: {img_name}")

if dry_run:
    print("\n✅ Dry-run 완료. dry_run=False 로 설정 후 다시 실행하면 실제로 복사됩니다.")
else:
    print(f"\n✅ 경로에 복사되었습니다.")

# 모델 학습

보강된 데이터셋을 이용하여 모델 파인튜닝

In [ ]:
from ultralytics import YOLO

# 1) 설정
DATA_YAML  = "tree/Datasets/250630_split/data.yaml"
INIT_MODEL = "runs/train/Finetuned_413/weights/best.pt"
OUTPUT_DIR = "runs/train"
EPOCHS     = 300
LR0        = 1e-4
LRF        = 0.01
BATCH_SIZE = 8
IMG_SIZE   = 640

# 2) 모델 로드
model = YOLO(INIT_MODEL)

# 3) fine-tuning 시작
result = model.train(
    data         = DATA_YAML,
    epochs       = EPOCHS,
    batch        = BATCH_SIZE,
    imgsz        = IMG_SIZE,
    lr0          = LR0,
    lrf          = LRF,
    warmup_epochs= 5,
    optimizer    ='AdamW',
    augment      = True,
    hsv_h        = 0.015,       # 색상 변형 허용 폭
    hsv_s        = 0.1,
    hsv_v        = 0.1,
    degrees      = 10.0,        # 회전 ±10°
    translate    = 0.1,         # 평행 이동 ±10%
    scale        = 0.3,         # 스케일 0.5~1.5
    shear        = 2.0,         # 시어링 ±2°
    patience     = 40,
    project      = OUTPUT_DIR,
    name         = "Finetuned_428"
)

print(f"\n🎉 모델 학습 완료! 결과는 {result.save_dir} 에 저장되었습니다.")

# 모델 성능 지표 확인하기

In [ ]:
from ultralytics import YOLO

MODEL_PATH = "runs/train/Finetuned_428/weights/best.pt"
DATA_PATH = "tree/Datasets/250630_split/data.yaml"

model = YOLO(MODEL_PATH)
metrics = model.val(data=DATA_PATH)

print("mAP50-95:", metrics.box.map)       # mAP@0.5-0.95
print("mAP50:", metrics.box.map50)        # mAP@0.5
print("Precision:", metrics.box.mp)      # Mean Precision
print("Recall:", metrics.box.mr)         # Mean Recall

# 특정 클래스 미탐 확인

나무 이미지에서는 클래스를 몇개 검출했냐 못했냐는 크게 중요하지 않다 판단,
대신 tree 클래스는 무조건 있을 것이므로(size, loc 속성에도 필요함)
tree 클래스를 0개 검출한 이미지를 색출함.

In [ ]:
import os
import shutil
from tqdm import tqdm
from ultralytics import YOLO
from utils import GetModelPredictResult

# ──────────────────────────────────────
# 설정
# ──────────────────────────────────────
MODEL_PATH    = "runs/train/Finetuned_428/weights/best.pt"
SRC_DIR       = "tree/train"
DST_DIR       = "tree/Prediction/tree_0_detected"
CONF_THRESH   = 0.5
IOU_THRESH    = 0.5
IMG_EXTS      = (".jpg", ".png")

# 학습에 사용된 이미지 폴더들
TRAIN_USED_DIRS = [
    "tree/Datasets/250630_split/images/train",
    "tree/Datasets/250630_split/images/val",
]

# ──────────────────────────────────────
# 준비
# ──────────────────────────────────────
model = YOLO(MODEL_PATH)

# names dict 을 뒤집어서 {class_name: class_idx} 맵 생성
name_to_idx = {v: k for k, v in model.names.items()}
tree_cls_idx = name_to_idx["tree"]

# 이미 학습에 쓴 파일명 세트 생성
used_fns = set()
for d in TRAIN_USED_DIRS:
    for fn in os.listdir(d):
        if fn.lower().endswith(IMG_EXTS):
            used_fns.add(fn)

os.makedirs(DST_DIR, exist_ok=True)

imageCount = 0

for fn in tqdm(os.listdir(SRC_DIR), desc="Scanning images"):
    if not fn.lower().endswith(IMG_EXTS):
        continue

    # 학습셋에 이미 포함된 파일은 건너뛰기
    if fn in used_fns:
        continue

    img_path = os.path.join(SRC_DIR, fn)
    res = GetModelPredictResult(model, img_path)

    clses = res.boxes.cls.cpu().numpy()
    # tree 클래스가 하나도 없으면 복사
    if not (clses == tree_cls_idx).any():
        shutil.copy2(img_path, os.path.join(DST_DIR, fn))
        imageCount += 1

print(f"✅ tree 클래스를 못잡은 이미지 {imageCount} 장이 복사 완료되었습니다.")


# 모델 예측 확인

현재 학습시킨 모델이 train 이미지 10080장을 예측한 이미지 파일을 복사하여
모델이 예측하는 경향을 파악하고, 후처리에 대한 전략을 세우기 위함

In [ ]:
from tqdm import tqdm
import os
import cv2
from ultralytics import YOLO
from utils import GetModelPredictResult

def visualize_predictions(model, img_paths, save_dir="visualizations"):
    """
    Run inference on a list of images and save annotated results to a single directory with progress bar.
    
    Args:
        model:      YOLO model instance
        img_paths:  list of image file paths
        save_dir:   folder to save annotated images
    """
    os.makedirs(save_dir, exist_ok=True)

    for img_path in tqdm(img_paths, desc="Visualizing images"):

        # run prediction
        res = GetModelPredictResult(model, img_path)

        # plot annotated image (all detections)
        annotated = res.plot()  # numpy RGB array
        bgr = cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR)
        fname = os.path.basename(img_path)
        
        # save annotated image to single folder
        cv2.imwrite(os.path.join(save_dir, fname), bgr)
    
    print(f"✅ All visualizations saved to: {save_dir}")

model = YOLO("runs/train/Finetuned_428/weights/best.pt")
img_dir = "tree/train"
img_paths = [
    os.path.join(img_dir, f) for f in os.listdir(img_dir)
    if f.lower().endswith(('.jpg', '.png'))
]

visualize_predictions(
    model, img_paths,
    save_dir="review/Visualizations_all"
)


# tree_test.csv 파일 생성

In [ ]:
import os
import cv2
import math
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from utils import GetModelPredictResult

# ───────────────────────────────────────────────────
# 1) 환경 설정
# ───────────────────────────────────────────────────
MODEL_PATH     = "runs/train/Finetuned_428/weights/best.pt"
TEST_DIR       = "tree/test"    # 예측할 이미지 폴더
OUTPUT_CSV     = "tree_test.csv"  # 저장할 결과 파일명
IMG_EXTS       = (".jpg", ".png")
LOC_THRESHOLDS = {
    "left":   (0.0,  0.33),
    "center": (0.33, 0.66),
    "right":  (0.66, 1.0),
}
SIZE_THRESHOLDS = {
    "small":  (0.0,  0.1),   # 박스 면적/전체 면적
    "middle": (0.1, 0.3),
    "big":  (0.3,  1.0),
}
CLASS_NAMES = ["branch", "root", "crown", "fruit", "gnarl"]

# ───────────────────────────────────────────────────
# 2) 모델 로드
# ───────────────────────────────────────────────────
model = YOLO(MODEL_PATH)

# ───────────────────────────────────────────────────
# 3) 예측할 이미지 목록
# ───────────────────────────────────────────────────
imgs = sorted([f for f in os.listdir(TEST_DIR) if f.lower().endswith(IMG_EXTS)])

# ───────────────────────────────────────────────────
# 4) 결과 담을 리스트
# ───────────────────────────────────────────────────
rows = []

# ───────────────────────────────────────────────────
# 5) 예측 실행
# ───────────────────────────────────────────────────
for img_name in tqdm(imgs, desc="Processing test images"):
    img_path = os.path.join(TEST_DIR, img_name)
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    area_total = w * h

    # 5.1) 모델 예측
    res = GetModelPredictResult(model, img_path)
    xyxy  = res.boxes.xyxy.cpu().numpy()  # N×4 배열
    clses = res.boxes.cls.cpu().numpy()   # N 길이

    # 5.2) 클래스별 y/n 플래그
    flags = {c: "n" for c in CLASS_NAMES}
    for cls in clses:
        name = model.names[int(cls)]
        if name in flags:
            flags[name] = "y"

    # 5.3) tree 박스 위치/크기 계산
    tree_boxes = [
        (x1, y1, x2, y2)
        for (x1, y1, x2, y2), cls in zip(xyxy, clses)
        if model.names[int(cls)] == "tree"
    ]
    loc, size = "", ""
    if tree_boxes:
        # 면적으로 가장 큰 박스 선택
        areas = [((x2 - x1) * (y2 - y1), (x1, y1, x2, y2)) for (x1, y1, x2, y2) in tree_boxes]
        max_area, box = max(areas, key=lambda x: x[0])
        x1, y1, x2, y2 = box
        cx = ((x1 + x2) / 2) / w  # [0,1] 범위

        # loc 결정
        for key, (lo, hi) in LOC_THRESHOLDS.items():
            if lo <= cx < hi:
                loc = key
                break

        # size 결정
        ratio = max_area / area_total
        for key, (lo, hi) in SIZE_THRESHOLDS.items():
            if lo <= ratio < hi:
                size = key
                break

    # 5.4) 한 행으로 합치기
    row = {
        "id":        os.path.splitext(img_name)[0],
        "branch_yn": flags["branch"],
        "root_yn":   flags["root"],
        "crown_yn":  flags["crown"],
        "fruit_yn":  flags["fruit"],
        "gnarl_yn":  flags["gnarl"],
        "loc":       loc,
        "size":      size,
    }
    rows.append(row)

# ───────────────────────────────────────────────────
# 6) DataFrame → CSV 저장
# ───────────────────────────────────────────────────
df = pd.DataFrame(rows)

# encoding="utf-8-sig" 인자가 없으면 한글이 깨짐
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print(f"✅ 예측 결과를 {OUTPUT_CSV}에 저장했습니다.")
